# Assignment 3

In [1]:
# Import required libraries
import os
import sys
import gzip
import json
from pathlib import Path
import csv
import difflib
import pandas as pd
import pyarrow as pa
from pyarrow.json import read_json
import pyarrow.parquet as pq
from fastavro import writer, reader, parse_schema
import pygeohash
import snappy
import jsonschema
from jsonschema.exceptions import ValidationError
from genson import SchemaBuilder

In [2]:
# function to read and load jsonl zip file

endpoint_url='https://storage.budsc.midwest-datascience.com'

current_dir = Path(os.getcwd()).absolute()
schema_dir = current_dir.joinpath('schemas')
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)

# read file from aws s3 bucket

def read_jsonl_data_aws():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
        

    return records

# read file from local directory

def read_jsonl_data():
    
    file_path = 'C:/Users/Supraja/Documents/GitHub/dsc650/data/processed/openflights/routes.jsonl.gz'
    
    with gzip.open(file_path,'rb') as f:
        records = [json.loads(line) for line in f.readlines()]
        return records


Load the records from https://storage.budsc.midwest-datascience.com/data/processed/openflights/routes.jsonl.gz 

In [3]:
records = read_jsonl_data()

## 3.1

### 3.1.a JSON Schema

In [4]:
# create json Schema

def create_json_schema():
    
    schema_path = schema_dir.joinpath('routes-schema.json')
    builder = SchemaBuilder(schema_uri=None)
    records = read_jsonl_data()   # read json file
    # records_str = json.dumps(records)
    builder.add_object(records[0])
    json_schema = builder.to_schema()
    
    with open(schema_path,'w') as f:
        schema = json.dumps(json_schema)
        f.write(schema)
        
# run function to create json schema        
create_json_schema()

In [5]:
def validate_jsonl_data(records):
    schema_path = schema_dir.joinpath('routes-schema.json')
    
    with open(schema_path,'r') as f:
        schema = json.load(f)
        
    
    validation_csv_path = str(current_dir)+'/results/validations.csv'
    
    with open(validation_csv_path, 'w') as f:
        # add header to csv file
        writer = csv.writer(f)
        writer.writerow(['record_number','error'])
        
        for i, record in enumerate(records):
            try:
                jsonschema.validate(instance=record,schema=schema)
                
            except ValidationError as e:
                # write error to validations.csv file
                writer = csv.writer(f)
                writer.writerow([i,e])


In [6]:
records = read_jsonl_data()
validate_jsonl_data(records)

### 3.1.b Avro

In [7]:
def create_avro_dataset(records):
    schema_path = schema_dir.joinpath('routes.avsc')
    data_path = results_dir.joinpath('routes.avro')
    ## TODO: Use fastavro to create Avro dataset
    
    with open(schema_path, 'r') as f:
        schema = json.load(f)
        
    parsed_schema = parse_schema(schema)
    
    # write records to avro dataset
    
    with open(data_path, 'wb') as f_out:
        writer(f_out,parsed_schema,records)
    
        
create_avro_dataset(records)

In [8]:
#  read data from avro data set, 1 record

data_path = results_dir.joinpath('routes.avro')

with open(data_path,'rb') as f_read:
    for record in reader(f_read):
        print(record)
        break

{'airline': {'airline_id': 410, 'name': 'Aerocondor', 'alias': 'ANA All Nippon Airways', 'iata': '2B', 'icao': 'ARD', 'callsign': 'AEROCONDOR', 'country': 'Portugal', 'active': True}, 'src_airport': {'airport_id': 2965, 'name': 'Sochi International Airport', 'city': 'Sochi', 'iata': 'AER', 'icao': 'URSS', 'latitude': 43.449902, 'longitude': 39.9566, 'timezone': 3.0, 'dst': 'N', 'tz_id': 'Europe/Moscow', 'type': 'airport', 'source': 'OurAirports'}, 'dst_airport': {'airport_id': 2990, 'name': 'Kazan International Airport', 'city': 'Kazan', 'iata': 'KZN', 'icao': 'UWKD', 'latitude': 55.606201171875, 'longitude': 49.278701782227, 'timezone': 3.0, 'dst': 'N', 'tz_id': 'Europe/Moscow', 'type': 'airport', 'source': 'OurAirports'}, 'codeshare': False, 'stops': 0, 'equipment': ['CR2']}


### 3.1.c Parquet

In [9]:
# create parquet dataset

def create_parquet_dataset():
    parquet_output_path = results_dir.joinpath('routes.parquet')
    
    file_path = 'C:/Users/Supraja/Documents/GitHub/dsc650/data/processed/openflights/routes.jsonl.gz'
    
    with gzip.open(file_path,'rb') as f:
        table = read_json(f)
        pq.write_table(table,parquet_output_path)
        
        
create_parquet_dataset()

In [10]:
#  read data from parquet datset

parquet_output_path = results_dir.joinpath('routes.parquet')

with open(parquet_output_path,'rb') as f_pq:
    table_pq = pq.read_table(f_pq)
    
    df = table_pq.to_pandas()
    
df.head(2)

airline  \
0  {'airline_id': 410, 'name': 'Aerocondor', 'ali...   
1  {'airline_id': 410, 'name': 'Aerocondor', 'ali...   

                                         src_airport  \
0  {'airport_id': 2965.0, 'name': 'Sochi Internat...   
1  {'airport_id': 2966.0, 'name': 'Astrakhan Airp...   

                                         dst_airport  codeshare equipment  
0  {'airport_id': 2990.0, 'name': 'Kazan Internat...      False     [CR2]  
1  {'airport_id': 2990.0, 'name': 'Kazan Internat...      False     [CR2]

### 3.1.d Protocol Buffers

In [11]:
sys.path.insert(0, os.path.abspath('routes_pb2'))

import routes_pb2

def _airport_to_proto_obj(airport):
    obj = routes_pb2.Airport()
    if airport is None:
        return None
    if airport.get('airport_id') is None:
        return None

    obj.airport_id = airport.get('airport_id')
    if airport.get('name'):
        obj.name = airport.get('name')
    if airport.get('city'):
        obj.city = airport.get('city')
    if airport.get('iata'):
        obj.iata = airport.get('iata')
    if airport.get('icao'):
        obj.icao = airport.get('icao')
    if airport.get('altitude'):
        obj.altitude = airport.get('altitude')
    if airport.get('timezone'):
        obj.timezone = airport.get('timezone')
    if airport.get('dst'):
        obj.dst = airport.get('dst')
    if airport.get('tz_id'):
        obj.tz_id = airport.get('tz_id')
    if airport.get('type'):
        obj.type = airport.get('type')
    if airport.get('source'):
        obj.source = airport.get('source')

    obj.latitude = airport.get('latitude')
    obj.longitude = airport.get('longitude')

    return obj


def _airline_to_proto_obj(airline):
    obj = routes_pb2.Airline()
    if not airline.get('name'):
        return None
    if not airline.get('airline_id'):
        return None
    
    obj.airline_id = airline.get('airline_id')
    
    if airline.get('name'):
        obj.name = airline.get('name')
    if airline.get('alias'):
        obj.alias = airline.get('alias')
    if airline.get('iata'):
        obj.iata = airline.get('iata')
    if airline.get('icao'):
        obj.icao = airline.get('icao')
    if airline.get('callsign'):
        obj.callsign = airline.get('callsign')
    obj.active = airline.get('active')
    if airline.get('country'):
        obj.country = airline.get('country')
    
    return obj


def create_protobuf_dataset(records):
    routes = routes_pb2.Routes()
    for record in records:
        route = routes_pb2.Route()
        airline = _airline_to_proto_obj(record.get('airline', {}))
        if airline:
            route.airline.CopyFrom(airline)
        src_airport = _airport_to_proto_obj(record.get('src_airport', {}))
        if src_airport:
            route.src_airport.CopyFrom(src_airport)
        dst_airport = _airport_to_proto_obj(record.get('dst_airport', {}))
        if dst_airport:
            route.dst_airport.CopyFrom(dst_airport)
            
        route.codeshare = record.get('codeshare')
        
        route.stops = record.get('stops',0)
        eqp = record.get('equipment',[])
        route.equipment.append(b'eqp')

        routes.route.append(route)

    data_path = results_dir.joinpath('routes.pb')

    with open(data_path, 'wb') as f:
        f.write(routes.SerializeToString())
        
    compressed_path = results_dir.joinpath('routes.pb.gzip')
    
    with open(compressed_path, 'wb') as f:
        f.write(gzip.compress(routes.SerializeToString()))
        
    compressed_path = results_dir.joinpath('routes.pb.snappy')
    
    with open(compressed_path, 'wb') as f:
        f.write(snappy.compress(routes.SerializeToString()))
    
    data_path = results_dir.joinpath('routes.avro')

    with open(data_path, 'wb') as f:
        f.write(routes.SerializeToString())
        
    compressed_path = results_dir.joinpath('routes.avro.gzip')
    
    with open(compressed_path, 'wb') as f:
        f.write(gzip.compress(routes.SerializeToString()))
    
    compressed_path = results_dir.joinpath('routes.avro.snappy')
    
    with open(compressed_path, 'wb') as f:
        f.write(snappy.compress(routes.SerializeToString()))
    
        data_path = results_dir.joinpath('routes.parquet')

    with open(data_path, 'wb') as f:
        f.write(routes.SerializeToString())
        
    compressed_path = results_dir.joinpath('routes.parquet.gzip')
    
    with open(compressed_path, 'wb') as f:
        f.write(gzip.compress(routes.SerializeToString()))
    
    compressed_path = results_dir.joinpath('routes.parquet.snappy')
    
    with open(compressed_path, 'wb') as f:
        f.write(snappy.compress(routes.SerializeToString()))
        

In [12]:
# create dataset using protocol buffers

records = read_jsonl_data()

create_protobuf_dataset(records)

### 3.1.e Output Sizes

In [13]:
#  get the size of each and store in list

file_format = ['avro','parquet','pb']

uncomp_file_names = ['routes.avro','routes.parquet','routes.pb']

gzip_file_names = ['routes.avro.gzip','routes.parquet.gzip','routes.pb.gzip']

snappy_file_names = ['routes.avro.snappy','routes.parquet.snappy','routes.pb.snappy']

uncomp_size = []

gzip_size = []

snappy_size = []

for file in uncomp_file_names:
    size = os.path.getsize(results_dir.joinpath(file))
    size = round(size/1024,2)
    uncomp_size.append(size)
    
for file in gzip_file_names:
    size = os.path.getsize(results_dir.joinpath(file))
    size = round(size/1024,2)
    gzip_size.append(size)
    
for file in snappy_file_names:
    size = os.path.getsize(results_dir.joinpath(file))
    size = round(size/1024,2)
    snappy_size.append(size)

In [14]:
# populate it in csv file

comparison_csv_path = results_dir.joinpath('comparison.csv') 

with open(comparison_csv_path, 'w') as f:
    headers = ['format','uncompressed','gzip','snappy']
    writer = csv.writer(f)
    writer.writerow(headers)
    for idx in range(3):
         writer.writerow([file_format[idx],uncomp_size[idx],gzip_size[idx],snappy_size[idx]])


In [15]:
#  display the comparison file

compare = pd.read_csv(comparison_csv_path)
compare

format  uncompressed     gzip   snappy
0     avro      22002.38  2040.44  3503.77
1  parquet      22002.38  2040.44  3503.77
2       pb      22002.38  2040.44  3503.77

## 3.2

### 3.2.a Simple Geohash Index

In [16]:
def create_hash_dirs(records):
    geoindex_dir = results_dir.joinpath('geoindex')
    geoindex_dir.mkdir(exist_ok=True, parents=True)
    hashes = []
    
    for record in records:
        src_airport = record.get('src_airport',{})
        if src_airport:
            lat = src_airport.get('latitude')
            long = src_airport.get('longitude')
        if lat and long:
            geoh = pygeohash.encode(lat,long)
            hashes.append(geoh)
            record['geohash'] = geoh
    
    hashes.sort()
    three_letter = sorted(list(set([entry[:3] for entry in hashes])))  # keep first three letter from encoded geohash w/ precision 12
    hash_index = {value: [] for value in three_letter}
    for record in records:
        geohash = record.get('geohash')
        if geohash:
            hash_index[geohash[:3]].append(record)

    for key, values in hash_index.items():
        output_dir = geoindex_dir.joinpath(str(key[:1])).joinpath(str(key[:2]))
        output_dir.mkdir(exist_ok=True, parents=True)
        output_path = output_dir.joinpath('{}.jsonl.gz'.format(key))
        with gzip.open(output_path, 'w') as f:
            json_output = '\n'.join([json.dumps(value) for value in values])
            f.write(json_output.encode('utf-8'))
    

In [17]:
# create hash directory

create_hash_dirs(records)

### 3.2.b Simple Search Feature

In [18]:
# function to get sorted index as dict obj

def get_sorted_index(data):   
    geoindex_dir = results_dir.joinpath('geoindex')
    geoindex_dir.mkdir(exist_ok=True, parents=True)
    hashes = []
    
    for record in data:
        src_airport = record.get('src_airport',{})
        if src_airport:
            lat = src_airport.get('latitude')
            long = src_airport.get('longitude')
        if lat and long:
            geoh = pygeohash.encode(lat,long)
            hashes.append(geoh)
            record['geohash'] = geoh
    
    hashes.sort()
    three_letter = sorted(list(set([entry[:3] for entry in hashes])))  # keep first three letter from encoded geohash w/ precision 12
    hash_index = {value: [] for value in three_letter}
    for record in data:
        geohash = record.get('geohash')
        if geohash:
            hash_index[geohash[:3]].append(record)
    return hash_index

In [19]:
#  create function to calculate approx distance and get short distance

def short_distance(geoh_code, match_code_list):
    distances = {}
    
    for code in match_code_list:
        distance = pygeohash.geohash_approximate_distance(geoh_code, code)
        distances[code] = round(distance/1000,4)
    sorted_dist = dict(sorted(distances.items(),key=lambda item:item[1]))
    return [list(sorted_dist.keys())[0],list(sorted_dist.values())[0]]

# function to search airport based on short distance from provided latitude and longitude

def airport_search(latitude, longitude):
    
    geohash = pygeohash.encode(latitude,longitude)
    
    new_code = geohash[:3]  
    
   
    airlines_data = read_jsonl_data()
    hashed_idx = {}
    hashed_idx = get_sorted_index(airlines_data)
    
    keys_list = list(hashed_idx.keys())
    partial_match_codes = []
    single_match_codes = []
    
    if new_code in keys_list:
        
        value = hashed_idx[new_code]
            
        src_airport = value[0].get('src_airport',{})
        name = src_airport.get('name')
        distance = pygeohash.geohash_approximate_distance(geohash,geohash)
        distance = round(distance/1000,2)
        print(f"The Name of Airport is: {name} and Given coordinates are direct Location of Airport")
    else:
        for key, val in hashed_idx.items():
            seq=difflib.SequenceMatcher(None, new_code,key)
            matched_str = thre[match.a: match.a + match.size]
            if len(matched_str) > 1:
                partial_match_codes.append(key)
            elif len(matched_str) == 1:
                single_match_codes.append(key)
            else:
                continue
        if len(partial_match_codes)>0:
        
            short_dist_values = short_distance(new_code, partial_match_codes)
        
            value = hashed_idx[short_dist_values[0]]
            src_airport = value[0].get('src_airport',{})
            name = src_airport.get('name')
            distance = short_dist_values[1]
            print(f"The Name of Nearest Airport is: {name} and approx. distance from given cordinates is: {distance} KM")
    
        elif len(single_match_codes)>0:
        
            short_dist_values = short_distance(new_code, partial_match_codes)
        
            value = hashed_idx[short_dist_values[0]]
            src_airport = value[0].get('src_airport',{})
            name = src_airport.get('name')
            distance = short_dist_values[1]
            print(f"The Name of Nearst Airport is: {name} and approx. distance from given cordinates is: {distance} KM")
        
        else:
        
            print('Please update latitude and longitude values and re-try')
            


In [20]:
airport_search(41.1499988, -95.91779)

The Name of Airport is: Eppley Airfield and Given coordinates are direct Location of Airport


In [21]:
#  test the function for different airport
airport_search(19.0931,72.8568)

The Name of Airport is: Chhatrapati Shivaji International Airport and Given coordinates are direct Location of Airport
